In [133]:
%reset -f
%load_ext autoreload
%autoreload 2

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import lsh_hyperplane as lsh
import math
import random
from nltk.util import ngrams

data = lsh.load_data("data/capture20110818.binetflow")
pdata = lsh.pre_process(data)

# We first discretize the values for the features we selected in the discretization task. 
# We then make trigrams 
# Optimal number of bins for source address, based on elbow method
features = ["Protocol"]
nbins = [3]

# Descretize only one infected host (with most flows)
discretized = pd.DataFrame()
discretized["StartTime"] = pdata["StartTime"].copy()
discretized['StartTime'] = discretized['StartTime'].str[:-7]
discretized["SourceAddress"] = pdata["SourceAddress"].copy()
discretized["DestinationAddress"] = pdata["DestinationAddress"].copy()
pdata["Protocol"] = lsh.encode_feature(pdata["Protocol"])

for feature, nbin in zip(features,nbins):
    discretized[feature], binsedges_infected = lsh.discretize_feature(pdata, feature, nbin, "kmeans")
    
# Remove pairs with less than three occurrences (unable to form 3-gram)
ip_pairs = discretized.groupby(['SourceAddress','DestinationAddress']).size().reset_index().rename(columns={0:'count'})

# for row in ip_pairs.iterrows():
#     if row[1]["count"] < 3:
#         discretized = discretized.loc[~((discretized["SourceAddress"] == row[1].SourceAddress) & (discretized["DestinationAddress"] == row[1].DestinationAddress))]




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [323]:
## WORKING 

ip_pairs = discretized.groupby(["SourceAddress","DestinationAddress"]).size().reset_index()

column_names = list()
column_names.append("SourceAddress")
column_names.append("DestinationAddress")
column_names.append("trigrams")

n_projections = 3

for i in range(n_projections):
    column_names.append("hash_code_"+str(i))

# Create trigrams for each ip_pair
ip_pairs_trigrams = pd.DataFrame(columns=column_names)

for pair in ip_pairs.iterrows():
    source = pair[1].SourceAddress
    dest = pair[1].DestinationAddress
    
    filtered = discretized.loc[(discretized["SourceAddress"] == source) & (discretized["DestinationAddress"] == dest)]
    trigrams = list(set(ngrams(filtered["Protocol"].astype(str),3)))
    
    ip_pairs_trigrams = ip_pairs_trigrams.append({"SourceAddress": source, "DestinationAddress": dest, "trigrams": trigrams}, ignore_index=True)

hash_size = 3
dimension = 3

random_hyperplanes = list()

for i in range(n_projections):
    hyperplanes = np.random.randn(hash_size,dimension)
    random_hyperplanes.append(hyperplanes)
    print(hyperplanes)

# Every pair in the list
for pair_id in range(len(ip_pairs_trigrams)):
    pair = ip_pairs_trigrams.iloc[pair_id]
    pair_trigrams = pair.trigrams
    hash_codes = list()
    
    # For every trigram in the list with unique trigrams for this pair
    for i in range(n_projections):
        hash_codes = list()
        for trigram in pair_trigrams:
            code = list()
            
            #For every random hyperplane we calculate the hash and add it to the list of all hash codes
            for hyperplane in random_hyperplanes[i]:
                code.append(lsh.sign_function(lsh.dot_product(trigram,hyperplane)))
            
            hash_codes.append(code)
            
        ip_pairs_trigrams["hash_code_"+str(i)].iloc[pair_id] = hash_codes

ip_pairs_trigrams.head(110)

KeyboardInterrupt: 

In [353]:
## BINS

ip_pairs = discretized.groupby(["SourceAddress","DestinationAddress"]).size().reset_index()

column_names = list()
column_names.append("SourceAddress")
column_names.append("DestinationAddress")
column_names.append("trigrams_counts")

possible_trigrams = list(set(ngrams(discretized["Protocol"].astype(str),3)))

n_projections = 5

for i in range(n_projections):
    column_names.append("hash_code_"+str(i))
    
column_names.append("bin")

# Create trigrams for each ip_pair
ip_pairs_trigrams = pd.DataFrame(columns=column_names)

for pair in ip_pairs.iterrows():
    source = pair[1].SourceAddress
    dest = pair[1].DestinationAddress
    counts = [0]*len(possible_trigrams)
    
    filtered = discretized.loc[(discretized["SourceAddress"] == source) & (discretized["DestinationAddress"] == dest)]
    trigrams = list(ngrams(filtered["Protocol"].astype(str),3))
    
    for trigram in trigrams:
        counts[possible_trigrams.index(trigram)] += 1
    
    ip_pairs_trigrams = ip_pairs_trigrams.append({"SourceAddress": source, "DestinationAddress": dest, "trigrams_counts": counts}, ignore_index=True)
    
bin_size = 2
dimension = len(possible_trigrams)

random_hyperplanes = list()

for i in range(n_projections):
    hyperplanes = np.random.randn(bin_size,dimension)
    random_hyperplanes.append(hyperplanes)

# Every pair in the list
for pair_id in range(len(ip_pairs_trigrams)):
    pair = ip_pairs_trigrams.iloc[pair_id]
    
    hash_sum = [0]*bin_size
    
    hashed_bins = list()
    
    # For every trigram in the list with unique trigrams for this pair
    for i in range(n_projections):
        hash_bin = ""
            
        #For every random hyperplane we calculate the hash and add it to the list of all hash codes
        for hyperplane in random_hyperplanes[i]:
            hash_bin += str(lsh.sign_function(lsh.dot_product(pair.trigrams_counts,hyperplane)))
            
        ip_pairs_trigrams["hash_code_"+str(i)].iloc[pair_id] = hash_bin
        hashed_bins.append(int(hash_bin,2))
        
#     ip_pairs_trigrams["possible_bins"].iloc[pair_id] = [i for i, x in enumerate(hash_sum) if x == max(hash_sum)]
    ip_pairs_trigrams["bin"].iloc[pair_id] = hashed_bins
        
ip_pairs_trigrams.head(60)

,SourceAddress,DestinationAddress,trigrams_counts,hash_code_0,hash_code_1,hash_code_2,hash_code_3,hash_code_4,bin
0,147.32.1.20,147.32.85.20,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",00,00,00,00,00,"[0, 0, 0, 0, 0]"
1,147.32.1.20,147.32.85.7,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",11,00,00,00,10,"[3, 0, 0, 0, 2]"
2,147.32.1.20,147.32.85.8,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",00,00,00,00,00,"[0, 0, 0, 0, 0]"
3,147.32.1.20,147.32.86.135,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",00,00,00,00,00,"[0, 0, 0, 0, 0]"
4,147.32.1.20,147.32.86.155,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",00,00,00,00,00,"[0, 0, 0, 0, 0]"
5,147.32.1.20,147.32.86.168,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",11,00,00,00,10,"[3, 0, 0, 0, 2]"
6,147.32.1.20,147.32.86.176,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",00,00,00,00,00,"[0, 0, 0, 0, 0]"
7,147.32.1.20,147.32.86.77,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",00,00,00,00,00,"[0, 0, 0, 0, 0]"
8,147.32.80.9,147.32.84.118,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",11,00,00,00,10,"[3, 0, 0, 0, 2]"
9,147.32.80.9,147.32.84.123,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",00,00,00,00,00,"[0, 0, 0, 0, 0]"


In [354]:
# For obtaining the correct frequencies ip-pairs we neglect the ip pairs that have no trigrams, 
# e.g. have less than three flows between them. These hosts influence the results quite a lot.
# From these remove pairs a lot of them are infected hosts, which only have a connection with other 
# hosts once or twice.
non_zero_indices = list()
for i in range(len(ip_pairs_trigrams)):
    row = ip_pairs_trigrams.iloc[i]
    
    if np.sum(row.trigrams_counts) > 0:
        non_zero_indices.append(i)
        
res_ip_pairs = ip_pairs_trigrams.iloc[non_zero_indices]

# Retrieve infected and non infected source hosts. Infected host is 1, normal host is 0, unknown host is -1
res_ip_pairs["Infected"] = -1

for i in range(len(res_ip_pairs)):
    ip = res_ip_pairs.SourceAddress.iloc[i]
    if lsh.is_infected(ip):
        res_ip_pairs.Infected.iloc[i] = 1
    elif lsh.is_normal(ip):
        res_ip_pairs.Infected.iloc[i] = 0

infected = res_ip_pairs.loc[res_ip_pairs["Infected"] == 1]
normal = res_ip_pairs.loc[res_ip_pairs["Infected"] == 0]
unknown = res_ip_pairs.loc[res_ip_pairs["Infected"] == -1]

print("Infected host bins:")
print(infected.bin.value_counts()/len(infected)*100, "\n")
print("Normal host bins:")
print(normal.bin.value_counts()/len(normal)*100, "\n")
print("Unknown host bins:")
print(unknown.bin.value_counts()/len(unknown)*100, "\n")

Infected host bins:
[3, 0, 0, 0, 2]    64.516129
[1, 0, 2, 2, 3]    32.258065
[0, 1, 1, 1, 3]     3.225806
Name: bin, dtype: float64 

Normal host bins:
[0, 1, 1, 1, 3]    81.992337
[3, 0, 0, 0, 2]    16.091954
[1, 0, 2, 2, 3]     1.532567
[1, 0, 3, 2, 3]     0.383142
Name: bin, dtype: float64 

Unknown host bins:
[3, 0, 0, 0, 2]    90.909091
[0, 1, 1, 1, 3]     9.090909
Name: bin, dtype: float64 



/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [314]:
import random
import time
# Determine Euclidean distances between trigrams count vectors of every pair and select the distance of
# two random numbers
random_number_one = random.randint(0, len(res_ip_pairs))
random_number_two = random.randint(0, len(res_ip_pairs))

start = time.time()

distances_all = [[100]*len(res_ip_pairs)]*len(res_ip_pairs)
for i in range(len(res_ip_pairs)):
    pair = res_ip_pairs.trigrams_counts.iloc[i]
    for j in range(len(res_ip_pairs)):
        other_pair = res_ip_pairs.trigrams_counts.iloc[j]
        distances[i][j] = lsh.euclidean_distance(pair,other_pair)

end = time.time()

print("Distance between pair_id ",random_number_one, " and pair_id ",random_number_two)
print("Time elapsed",end-start)

# Determine distances between pairs which are hashed to the same bins
pair_one = res_ip_pairs.iloc[random_number_one]
pair_two = res_ip_pairs.iloc[random_number_two]
bins = pair_two.possible_bins

while bins != pair_one.possible_bins:
    random_number = random.randint(0,len(res_ip_pairs))
    pair_two = res_ip_pairs.iloc[random_number_one]
    
distance_bin = lsh.euclidean_distance()
        

for bin in pair.possible_bins:
    print(bin)

Time elapsed 2.609466791152954


In [315]:
print(distances)

[[3.1622776601683795, 1.4142135623730951, 4.123105625617661, 1.4142135623730951, 42.01190307520001, 3.1622776601683795, 16.0312195418814, 211.00236965494014, 1.4142135623730951, 3.1622776601683795, 5.0990195135927845, 5.0990195135927845, 2.23606797749979, 18.027756377319946, 16.0312195418814, 11.045361017187261, 4.123105625617661, 5.0990195135927845, 17.029386365926403, 22.02271554554524, 3.1622776601683795, 5.0990195135927845, 5.0990195135927845, 6.082762530298219, 9.055385138137417, 1.4142135623730951, 2.23606797749979, 4.123105625617661, 13.038404810405298, 9.055385138137417, 2.23606797749979, 9.055385138137417, 2.23606797749979, 9.055385138137417, 18.027756377319946, 29.017236257093817, 17.029386365926403, 1.4142135623730951, 10.04987562112089, 3.1622776601683795, 3.0, 575.0008695645598, 3.0, 0.0, 2.0, 9.0, 0.0, 0.0, 6.0, 1.0, 9.0, 0.0, 0.0, 4.0, 10.0, 33.0, 22.0, 2.0, 3.0, 0.0, 0.0, 26.0, 0.0, 2.0, 2.0, 8.0, 6.0, 5.0, 4.0, 1.0, 1.0, 1.0, 1.0, 3.0, 7.0, 4.0, 5.0, 0.0, 0.0, 2.0, 2.0

In [79]:
from nltk.util import ngrams

# Create trigrams for each ip_pair
unique_trigrams = pd.DataFrame(columns=["SourceAddress","DestinationAddress","trigrams","hash_codes"])

for pair in ip_pairs.iterrows():
    source = pair[1].SourceAddress
    dest = pair[1].DestinationAddress
    
    filtered = discretized.loc[(discretized["SourceAddress"] == source) & (discretized["DestinationAddress"] == dest)]
    trigrams = list(set(ngrams(filtered["Protocol"].astype(str),3)))
    
    unique_trigrams = unique_trigrams.append({"SourceAddress": source, "DestinationAddress": dest, "trigrams": trigrams}, ignore_index=True)

hash_size = 3
dimension = 3
n_projections = 1

for i in range(n_projections):
    random_hyperplanes = np.random.randn(hash_size,dimension)
    for i in range(len(unique_trigrams)):
        pair = unique_trigrams.iloc[i]
        
        hash_codes = list()
        
#         print(pair.trigrams)
        
        for trigram in pair.trigrams:
            code = list()
            for hyperplane in random_hyperplanes:
                code.append(lsh.sign_function(lsh.dot_product(trigram,hyperplane)))

            hash_codes.append(code)
            
            unique_trigrams.hash_codes.iloc[i] = hash_codes
    
#     discretized["hash_code_"+str(i)] = hash_codes
unique_trigrams.head(20)

,SourceAddress,DestinationAddress,trigrams,hash_codes
0,147.32.1.20,147.32.85.7,"[(2, 2, 2)]","[[1, 1, 0]]"
1,147.32.1.20,147.32.86.168,"[(2, 2, 2)]","[[1, 1, 0]]"
2,147.32.80.9,147.32.84.118,"[(2, 2, 2)]","[[1, 1, 0]]"
3,147.32.80.9,147.32.84.137,"[(2, 2, 2)]","[[1, 1, 0]]"
4,147.32.80.9,147.32.84.138,"[(2, 2, 2)]","[[1, 1, 0]]"
5,147.32.80.9,147.32.84.184,"[(2, 2, 2)]","[[1, 1, 0]]"
6,147.32.80.9,147.32.84.25,"[(2, 2, 2)]","[[1, 1, 0]]"
7,147.32.80.9,147.32.84.59,"[(2, 2, 2)]","[[1, 1, 0]]"
8,147.32.80.9,147.32.84.68,"[(2, 2, 2)]","[[1, 1, 0]]"
9,147.32.80.9,147.32.84.94,"[(2, 2, 2)]","[[1, 1, 0]]"


In [397]:
unique_trigrams.head(60)

,SourceAddress,DestinationAddress,trigrams,hash_codes
0,147.32.1.20,147.32.85.7,"[(2, 2, 2)]","[[0, 1, 0]]"
1,147.32.1.20,147.32.86.168,"[(2, 2, 2)]","[[0, 1, 0]]"
2,147.32.80.9,147.32.84.118,"[(2, 2, 2)]","[[0, 1, 0]]"
3,147.32.80.9,147.32.84.137,"[(2, 2, 2)]","[[0, 1, 0]]"
4,147.32.80.9,147.32.84.138,"[(2, 2, 2)]","[[0, 1, 0]]"
5,147.32.80.9,147.32.84.184,"[(2, 2, 2)]","[[0, 1, 0]]"
6,147.32.80.9,147.32.84.25,"[(2, 2, 2)]","[[0, 1, 0]]"
7,147.32.80.9,147.32.84.59,"[(2, 2, 2)]","[[0, 1, 0]]"
8,147.32.80.9,147.32.84.68,"[(2, 2, 2)]","[[0, 1, 0]]"
9,147.32.80.9,147.32.84.94,"[(2, 2, 2)]","[[0, 1, 0]]"


In [81]:
print(ip_pairs_trigrams.trigrams.iloc[106])

[('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2'), ('2', '2', '2

In [256]:
for i in range(len(test)):
    hyperplane_one = sign_function(dot_product(test[i],random_hyperplanes[0]))
    hyperplane_two = sign_function(dot_product(test[i],random_hyperplanes[1]))
    hyperplane_three = sign_function(dot_product(test[i],random_hyperplanes[2]))
    
    print(hyperplane_one,hyperplane_two,hyperplane_three)
    

0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 

1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 

0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
1 0 1
0 1 

1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
1 0 1
0 1 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
1 0 1
0 1 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 1
0 0 

0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 

...

0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
1 0 0
0 1 1
0 0 1
0 0 1
1 0 1
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 

0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 

1 0 0
0 1 1
0 0 1
0 0 1
1 0 1
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 

0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 

0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 

In [257]:
print(len(test))

121871


In [258]:
print(len(discretized))

121873


In [360]:
ip_pairs = discretized.groupby(["SourceAddress","DestinationAddress"]).size().reset_index()

In [373]:
unique_trigrams = pd.DataFrame(columns=["SourceAddress","DestinationAddress","trigrams"])

for pair in ip_pairs.iterrows():
    source = pair[1].SourceAddress
    dest = pair[1].DestinationAddress
    
    filtered = discretized.loc[(discretized["SourceAddress"] == source) & (discretized["DestinationAddress"] == dest)]
    trigrams = list(set(ngrams(filtered["Duration"].astype(str),3)))
    
    unique_trigrams = unique_trigrams.append({"SourceAddress": source, "DestinationAddress": dest, "trigrams": trigrams}, ignore_index=True)
    
#     print(unique_trigrams.trigrams.iloc[0])

In [372]:
filtered = discretized.loc[(discretized["SourceAddress"] == "147.32.80.9") & (discretized["DestinationAddress"] == "147.32.84.138")]
trigrams = list(set(ngrams(filtered["Protocol"].astype(str),3)))
filtered.head(60)

,StartTime,SourceAddress,DestinationAddress,Protocol,Duration,SourceBytes,trigrams,hash_code_0,hash_code_1,hash_code_2
486751,2011/08/18 12:19:17,147.32.80.9,147.32.84.138,2,0,0,"(2, 0, 0)","[0, 0, 0]","[1, 1, 0]","[1, 1, 0]"
494438,2011/08/18 12:23:13,147.32.80.9,147.32.84.138,2,0,0,"(2, 2, 1)","[1, 0, 0]","[1, 1, 0]","[1, 1, 0]"
494439,2011/08/18 12:23:13,147.32.80.9,147.32.84.138,2,0,0,"(2, 1, 2)","[1, 0, 0]","[1, 1, 0]","[0, 1, 0]"
494640,2011/08/18 12:23:21,147.32.80.9,147.32.84.138,2,0,0,"(2, 2, 2)","[1, 0, 0]","[1, 1, 0]","[1, 1, 0]"
494641,2011/08/18 12:23:21,147.32.80.9,147.32.84.138,2,0,0,"(2, 2, 2)","[1, 0, 0]","[1, 1, 0]","[1, 1, 0]"
494685,2011/08/18 12:23:23,147.32.80.9,147.32.84.138,2,0,0,"(2, 2, 2)","[1, 0, 0]","[1, 1, 0]","[1, 1, 0]"
494686,2011/08/18 12:23:23,147.32.80.9,147.32.84.138,2,0,0,"(2, 2, 2)","[1, 0, 0]","[1, 1, 0]","[1, 1, 0]"
495141,2011/08/18 12:23:43,147.32.80.9,147.32.84.138,2,0,0,"(2, 2, 2)","[1, 0, 0]","[1, 1, 0]","[1, 1, 0]"
495142,2011/08/18 12:23:43,147.32.80.9,147.32.84.138,2,0,0,"(2, 2, 1)","[1, 0, 0]","[1, 1, 0]","[1, 1, 0]"
497584,2011/08/18 12:25:08,147.32.80.9,147.32.84.138,2,0,0,"(2, 2, 2)","[1, 0, 0]","[1, 1, 0]","[1, 1, 0]"


In [367]:
ip_pairs.head(20)

,SourceAddress,DestinationAddress,0
0,147.32.1.20,147.32.85.7,5
1,147.32.1.20,147.32.86.168,3
2,147.32.80.9,147.32.84.118,6
3,147.32.80.9,147.32.84.137,3
4,147.32.80.9,147.32.84.138,44
5,147.32.80.9,147.32.84.184,5
6,147.32.80.9,147.32.84.25,18
7,147.32.80.9,147.32.84.59,213
8,147.32.80.9,147.32.84.68,3
9,147.32.80.9,147.32.84.94,5


In [370]:
discretized.Protocol

407        1
697        1
717        1
896        2
897        2
          ..
1309786    0
1309787    0
1309788    0
1309789    0
1309790    0
Name: Protocol, Length: 121873, dtype: category
Categories (3, int64): [0 < 1 < 2]

In [374]:
unique_trigrams.head(60)

,SourceAddress,DestinationAddress,trigrams
0,147.32.1.20,147.32.85.7,"[(0, 0, 0)]"
1,147.32.1.20,147.32.86.168,"[(0, 0, 0)]"
2,147.32.80.9,147.32.84.118,"[(0, 0, 0)]"
3,147.32.80.9,147.32.84.137,"[(0, 0, 0)]"
4,147.32.80.9,147.32.84.138,"[(0, 0, 0)]"
5,147.32.80.9,147.32.84.184,"[(0, 0, 0)]"
6,147.32.80.9,147.32.84.25,"[(0, 0, 0)]"
7,147.32.80.9,147.32.84.59,"[(0, 0, 0)]"
8,147.32.80.9,147.32.84.68,"[(0, 0, 0)]"
9,147.32.80.9,147.32.84.94,"[(0, 0, 0)]"


In [382]:
for pair in unique_trigrams.iterrows():
    trigrams = pair[1].trigrams
    for trigram in trigrams:
        print(trigram)

('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('3', '3', '3')
('3', '3', '2')
('3', '2', '3')
('3', '3', '3')
('3', '3', '2')
('2', '3', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('0', '0', '0')
('3', '3', '3')
('3', '3

In [28]:
df = ip_pairs_trigrams.copy()
df.head()

,SourceAddress,DestinationAddress,trigrams,hash_codes
0,147.32.1.20,147.32.85.7,"[(2, 2, 2)]","[[[0, 0, 0]], [[0, 0, 1]], [[1, 1, 0]]]"
1,147.32.1.20,147.32.86.168,"[(2, 2, 2)]","[[[0, 1, 0]], [[1, 1, 1]], [[0, 1, 0]]]"
2,147.32.80.9,147.32.84.118,"[(2, 2, 2)]","[[[0, 1, 1]], [[0, 0, 0]], [[1, 1, 1]]]"
3,147.32.80.9,147.32.84.137,"[(2, 2, 2)]","[[[1, 0, 1]], [[1, 0, 0]], [[0, 1, 0]]]"
4,147.32.80.9,147.32.84.138,"[(2, 2, 2)]","[[[1, 1, 0]], [[0, 0, 0]], [[1, 1, 0]]]"


In [32]:
df.insert('hash_codes_1',NaN,True)

NameError: name 'NaN' is not defined

In [37]:
ip_pairs_trigrams.SourceAddress.iloc[0]

'147.32.1.20'

[(0, 0, 1), (1, 0, 0), (0, 1, 0), (0, 0, 0)]


In [158]:
r1 = [-0.32292206, -0.65743537, -0.49250149, -0.51389714,  0.19468317,  0.50872072,
 -1.32818417,  0.74254978, -1.00664955,  0.09389382, -1.48139803, -1.02473662,
 -0.28337625,  0.24790388, -0.07295119,  1.66019368, -1.8681665,   0.40359731,
  0.39975404,  1.51688336,  0.68081489,  0.87034874,  0.45392065,  0.06086036,
  0.99237326, -0.15938439,  1.69842608]
r2 = [    1.17655703,  1.20882556, -0.10592746, -0.54468112, -0.64071041,  1.53070727,
         -0.18294368,  2.60249555,  1.55260772, -0.24358921,  0.23284924,  1.90368638,
         -2.23836685, -0.50368158, -0.12698003, -0.7513215,   0.61201654,  0.22015386,
         -0.22864834, -0.32992018, -1.50181643,  0.80022809,  1.3270992,   0.72829949,
         -0.59160117,  1.09468539,  0.77296184]
r3 = [-0.18854149,  0.12776587,  0.18534396,  2.99981876,  0.01693521, -1.54154611,
       1.05052648, -0.30529162,  1.85064023, -0.14920855,  1.19525313,  0.79809609,
       0.84363983,  0.42257033,  1.92680592, -2.16459662,  0.89468067,  0.35413835,
      -0.12357829, -0.55685461,  2.13052092, -0.43834173, -0.878121,   -0.63229906,
      -1.71637133, -1.08653568, -2.87395486]
v = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [161]:
lsh.sign_function(lsh.dot_product(v,r1))


0

0

In [318]:
ip_pairs_trigrams.head(60)

,SourceAddress,DestinationAddress,trigrams_counts,hash_code_0,hash_code_1,hash_code_2,hash_code_3,hash_code_4,hash_code_5,hash_code_6,hash_code_7,hash_code_8,hash_code_9,hash_code_10,hash_code_11,hash_code_12,hash_code_13,hash_code_14,possible_bins
0,147.32.1.20,147.32.85.20,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 1, 2]"
1,147.32.1.20,147.32.85.7,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 1]","[1, 0, 0]","[1, 0, 0]","[1, 1, 0]","[1, 0, 1]","[0, 1, 0]","[0, 1, 1]","[0, 0, 0]","[1, 1, 1]","[1, 1, 0]","[1, 0, 0]","[1, 0, 1]","[0, 1, 0]","[0, 1, 1]","[1, 1, 0]",[0]
2,147.32.1.20,147.32.85.8,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 1, 2]"
3,147.32.1.20,147.32.86.135,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 1, 2]"
4,147.32.1.20,147.32.86.155,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 1, 2]"
5,147.32.1.20,147.32.86.168,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 1]","[1, 0, 0]","[1, 0, 0]","[1, 1, 0]","[1, 0, 1]","[0, 1, 0]","[0, 1, 1]","[0, 0, 0]","[1, 1, 1]","[1, 1, 0]","[1, 0, 0]","[1, 0, 1]","[0, 1, 0]","[0, 1, 1]","[1, 1, 0]",[0]
6,147.32.1.20,147.32.86.176,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 1, 2]"
7,147.32.1.20,147.32.86.77,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 1, 2]"
8,147.32.80.9,147.32.84.118,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 1]","[1, 0, 0]","[1, 0, 0]","[1, 1, 0]","[1, 0, 1]","[0, 1, 0]","[0, 1, 1]","[0, 0, 0]","[1, 1, 1]","[1, 1, 0]","[1, 0, 0]","[1, 0, 1]","[0, 1, 0]","[0, 1, 1]","[1, 1, 0]",[0]
9,147.32.80.9,147.32.84.123,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 1, 2]"
